## 1. Setup Environment

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create project folder
import os
PROJECT_PATH = '/content/drive/MyDrive/QuantumAITrader'
os.makedirs(PROJECT_PATH, exist_ok=True)
os.chdir(PROJECT_PATH)

In [ ]:
# Install Dependencies
!pip install yfinance pandas numpy scikit-learn xgboost lightgbm ta-lib-binary
!pip install torch torchvision torchaudio
!pip install stable-baselines3 gymnasium shimmy
!pip install pysr deap
!pip install hmmlearn pyts

## 2. Load Codebase
Ensure you have uploaded the `core/` folder and `golden_architecture.py` to your Drive folder.

In [ ]:
# Verify files exist
!ls -R

## 3. Run Ticker Scanner (GPU Accelerated)

In [ ]:
import sys
sys.path.append(PROJECT_PATH)

# Import Scanner
from ticker_scanner import TickerScanner

# Initialize Scanner
scanner = TickerScanner()

# Run Scan on Full Watchlist
print("🚀 Starting GPU-Accelerated Scan...")
scanner.scan()  # Scans all 50 tickers

## 4. Analyze Results

In [ ]:
import json
import pandas as pd

# Load results
with open('scan_results.json', 'r') as f:
    results = json.load(f)

df = pd.DataFrame(results)

# Display Top Opportunities
print("🏆 TOP BUY SIGNALS")
display(df[df['signal'] == 'BUY'].sort_values('confidence', ascending=False).head(10))

print("\n📉 TOP SELL SIGNALS")
display(df[df['signal'] == 'SELL'].sort_values('confidence', ascending=False).head(10))